In [1]:
import os 
from collections import defaultdict


In [2]:
def extract_steps(line):
    
    #% FACT: program(program_id, program_name, qualified_program_name, begin_annotation_id, end_annotation_id).
    #print("Extracting steps.")
    ## Extract data between "(" and ")".
    data = line[line.index("(")+1:line.index(")")]
    
    data = data.split(',')
    #print(data)
    steps[data[0]] = {
                      "program_name" :data[1].strip().strip("'"),
                      "qualified_program_name": data[2].strip().strip("'")
                    }
    
    #print(steps)
    return

In [3]:
def extract_workflows(line):
    
    data = line[line.index("(")+1:line.index(")")]
    workflow[data[0]] = steps[data[0]]['program_name']
    return 

In [4]:
def extract_ports(line):
    #print("Extracting ports.")
    ## Extract data between "(" and ")".
    data = line[line.index("(")+1:line.index(")")]
    
    #% FACT: port(port_id, port_type, port_name, qualified_port_name, port_annotation_id, data_id).
    
    data = data.split(',')
    
    ports[int(data[0])] = {
                             "port_type": data[1].strip().strip("'").upper(),
                             "port_name": data[2].strip().strip("'"),
                             "qualified_port_name" : data[3].strip().strip("'"), 
                             "data_id":data[5].strip().strip("'")
                            }
    return 

In [5]:

def input_ports(line,in_ports):
    #print("Extracting Input ports.")
    
    ## Extract data between "(" and ")".
    ## % FACT: has_in_port(step_id, port_id).

    data = line[line.index("(")+1:line.index(")")]
    
    data = data.split(',')
    
    in_ports[data[0]].append((data[0].strip(), data[1].strip()))
    
    return

In [6]:
def output_ports(line,out_ports): 
    #print("Extracting output ports.")
    ## Extract data between "(" and ")".
    data = line[line.index("(")+1:line.index(")")]
    data = data.split(',')
    out_ports[data[0]].append((data[0].strip(), data[1].strip()))
    #print(data)
    return

In [7]:
def has_subprogram(line): 
    #print("Extracting output ports.")
    ## Extract data between "(" and ")".
    data = line[line.index("(")+1:line.index(")")]
    data = data.split(',')
    list1 = []
    #print(data[0],data[1])
    if data[0] in sub_programs.keys():
        list1 = sub_programs[data[0]]
        list1.append(data[1].strip())
        sub_programs[data[0]] = list1
    else:
        list1.append(data[1].strip())
        sub_programs[data[0]]= list1
    #print(data)
    return

In [8]:
steps = {}
ports = {}
workflow = {}
in_ports = defaultdict(list)
out_ports = defaultdict(list)
sub_programs = {}
cnt =0
with open("models/paleocar_models.P", "r") as yw_struct:
    line = yw_struct.readline()
    while line:
        if("program(" in line and "% FACT" not in line and "has_subprogram" not in line):
            extract_steps(line)
        elif("workflow(" in line and "% FACT" not in line):
            extract_workflows(line)            
        elif("has_subprogram(" in line and "% FACT" not in line):
            has_subprogram(line)            
        elif("port(" in line and "% FACT:" not in line and "has_in_port(" not in line and "has_out_port" not in line):
        #    print(line)
            extract_ports(line)
        elif("has_in_port(" in line and "% FACT" not in line):
            input_ports(line,in_ports)
        elif("has_out_port(" in line and "% FACT" not in line):
            output_ports(line,out_ports)
            
           
        line = yw_struct.readline()

In [9]:
sub_programs

{'1': ['2', '3', '4', '5', '6', '7', '11'], '7': ['8', '9', '10']}

In [10]:
def get_in_ports(program_id):
    #print("inside in ports")
    input_ports =[]
    for s in in_ports[program_id.strip()]:
        #print(s)
        input_ports.append(ports[int(s[1])]['port_name'])
        #print(ports[int(s[1])]['port_name'])
    return input_ports

In [11]:
def get_out_ports(program_id):
    output_ports =[]
    
    for s in out_ports[program_id.strip()]:
        output_ports.append(ports[int(s[1])]['port_name'])
    return output_ports

In [12]:
get_in_ports('3')

['itrdb', 'calibration_years', 'min_width']

In [13]:
in_ports

defaultdict(list,
            {'1': [('1', '1'),
              ('1', '2'),
              ('1', '3'),
              ('1', '4'),
              ('1', '5'),
              ('1', '6')],
             '10': [('10', '36'), ('10', '37')],
             '11': [('11', '39')],
             '2': [('2', '8')],
             '3': [('3', '9'), ('3', '10'), ('3', '11')],
             '4': [('4', '14'), ('4', '15'), ('4', '16')],
             '5': [('5', '18')],
             '6': [('6', '20'), ('6', '21')],
             '7': [('7', '23'), ('7', '24'), ('7', '25')],
             '8': [('8', '27'), ('8', '28'), ('8', '29')],
             '9': [('9', '32'), ('9', '33')]})

In [14]:
def write_file(filename, program_id):
    header = '''
#!/usr/bin/env cwl-runner


cwlVersion: v1.0
class: CommandLineTool
'''
    with open(filename, "w+") as writer:
        input_buffer = 'inputs: \n'
        for p in get_in_ports(program_id):
            input_buffer = input_buffer + ' '+ p + ': \n'+ '  type: string \n \n'

        #print(input_buffer)
        output_buffer = 'outputs: \n'
        output_buffer.strip()

        for p in get_out_ports(program_id):
            #print(len(p))
            if p =='' : 
                print("No output ports")
            else:
                output_buffer = output_buffer + ' '+ p + ': \n'+ '  type: string \n \n'

        #print (output_buffer)
        writer.write(header)    
        writer.write(input_buffer)
        writer.write(output_buffer)
        writer.write("\n")
        writer.write("baseCommand: ")
    
    return

In [15]:
def create_cwl_files(program_id): 
    filename = "cwl_files/"+ steps[program_id]['program_name'] + ".cwl"
    write_file(filename, program_id)
    
    return 


In [88]:
for wf in workflow:
    for sp in sub_programs[wf]:
        create_cwl_files(sp)

In [47]:
set(get_in_ports('5')).intersection(set(get_out_ports('4')))

{'reconstruction_matrix'}

In [55]:
a = set(get_in_ports('1')).intersection(set(get_in_ports('4')))

In [61]:
b = set(get_in_ports('7')).intersection(set(get_out_ports('4')))

In [62]:
b

set()

In [82]:
for sid in steps.keys(): 
    for id in range(int(sid),len(steps)):
        a = set(get_in_ports(str(id))).intersection(set(get_out_ports(str(sid))))
        if len(a) > 0 : 
            for elem in a: 
                print(steps[sid]["program_name"]+"/"+ elem, steps[str(id)]["program_name"] )

get_predictor_matrix/predictor_matrix get_carscores
get_predictor_matrix/max_preds calculate_Models
get_predictor_matrix/max_preds defineLinearModels
get_reconstruction_matrix/reconstruction_matrix get_predlist
get_predlist/predlist calculate_Models
get_predlist/predlist defineLinearModels
get_carscores/carscores calculate_Models
get_carscores/carscores defineLinearModels
defineLinearModels/matches calculateLinearModels
defineLinearModels/models calculateLinearModels
calculateLinearModels/model_errors simplifyLinearModels
calculateLinearModels/coefficients simplifyLinearModels


In [83]:
get_in_ports('6')

['prism_data_for_coordinates', 'predictor_matrix']